# Final Notebook: Data Preprocessing, Modelling, and Inference

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import sys
import os

In [2]:
# Setting path to load util functions
from pathlib import Path
parent_dir = Path.cwd().parents[0]
sys.path.append(os.path.abspath(parent_dir))

### Loading Train and Test Data

In [3]:
train_df = pd.read_csv('../../data/training_data.csv')
test_df = pd.read_csv('../../data/test_data.csv')

In [4]:
train_df['date_id'] = pd.to_datetime(train_df['date_id'])
test_df['date_id'] = pd.to_datetime(test_df['date_id'])

In [5]:
#Instantiating preprocessor object using DataPreprocessPipeline class
from utils.DataPreprocessPipeline import DataPreprocessPipeline
preprocessor = DataPreprocessPipeline(num_lags=3, rolling_window_size=3)

In [6]:
#merging train_df and test_df to preprocess the data together (so time-series features would be created)
full_df = pd.concat([train_df, test_df])

In [7]:
full_df

,date_id,item_dept,item_qty,net_sales,store,item,invoice_num
0,2021-11-01,Grocery,1.0,160.0,XYZ,16620,1475459.0
1,2021-11-01,Grocery,2.0,480.0,XYZ,32365,1475459.0
2,2021-11-01,Grocery,1.0,127.0,XYZ,31349,1475459.0
3,2021-11-01,Household,2.0,110.0,XYZ,1266,1475475.0
4,2021-11-01,Household,1.0,150.0,XYZ,114920,1475475.0
...,...,...,...,...,...,...,...
247619,2022-02-28,Grocery,5.0,500.0,XYZ,114873,NaN
247620,2022-02-28,Household,4.0,2361.0,XYZ,117497,NaN
247621,2022-02-28,Grocery,2.0,480.0,XYZ,77298,NaN
247622,2022-02-28,Beverages,1.0,1900.0,XYZ,1444,NaN


In [8]:
#preprocessing the data
full_df_processed = preprocessor.transform(full_df)

In [9]:
full_df_processed.head(20)

,date_id,item_dept,store,item_qty,net_sales,lag_item_qty_1,lag_item_qty_2,lag_item_qty_3,rolling_mean_item_qty_3,rolling_std_item_qty_3,cumsum_item_qty,cummean_item_qty,expanding_min_item_qty,expanding_max_item_qty,diff_item_qty,diff_item_qty_7,lag_net_sales_1,lag_net_sales_2,lag_net_sales_3,rolling_mean_net_sales_3,rolling_std_net_sales_3,cumsum_net_sales,cummean_net_sales,expanding_min_net_sales,expanding_max_net_sales,diff_net_sales,diff_net_sales_7,day_of_week,isWeekend,Is_Holiday,store_XYZ,item_dept_Grocery,item_dept_Household
42,2021-11-08,Beverages,ABC,974.000,246984.68200,937.000,884.000,991.000,937.333333,53.500779,6962.000,994.571429,837.000,1256.000,53.000,57.000,218668.693000,241963.000000,255695.365000,238775.686000,18717.981669,1.703852e+06,243407.457717,199145.596010,277904.838000,-23294.307000,57.000,0,0,0,0.0,0.0,0.0
43,2021-11-08,Beverages,XYZ,677.000,201386.08100,1043.000,1086.000,910.000,1013.000000,91.755109,6495.000,927.857143,779.000,1086.000,-43.000,-102.000,289993.000000,262426.080000,217746.000000,256721.693333,36459.734951,1.675697e+06,239385.355431,187931.769020,289993.000000,27566.920000,-102.000,0,0,0,1.0,0.0,0.0
44,2021-11-08,Grocery,ABC,2326.638,438841.43801,2646.820,2614.006,2654.582,2638.469333,21.538411,18100.925,2585.846429,2155.128,3021.684,32.814,-33.761,505512.742970,548660.900000,479383.663970,511185.768980,34985.300857,3.473914e+06,496273.373853,397946.965024,582206.592985,-43148.157030,-33.761,0,0,0,0.0,1.0,0.0
45,2021-11-08,Grocery,XYZ,2879.414,498037.79100,3608.962,3625.864,3287.828,3507.551333,190.473559,23689.611,3384.230143,2987.060,3625.864,-16.902,-107.646,661655.000010,622856.562004,557072.729000,613861.430338,52868.205452,4.010397e+06,572913.907431,491450.723000,661655.000010,38798.438006,-107.646,0,0,0,1.0,1.0,0.0
46,2021-11-08,Household,ABC,883.000,240294.89201,998.000,1152.000,986.000,1045.333333,92.570694,7114.000,1016.285714,819.000,1195.000,-154.000,-160.000,233781.880016,284326.000000,265589.692020,261232.524012,25552.215188,1.775416e+06,253630.794154,209100.513996,292020.462024,-50544.119984,-160.000,0,0,0,0.0,0.0,1.0
47,2021-11-08,Household,XYZ,944.000,237517.07200,1252.000,1315.000,963.000,1176.666667,187.702779,7906.000,1129.428571,948.000,1315.000,-63.000,-180.000,283841.000000,305140.577002,222311.011000,270430.862667,43012.301595,1.798679e+06,256954.189714,222311.011000,305140.577002,-21299.577002,-180.000,0,0,0,1.0,0.0,1.0
48,2021-11-09,Beverages,ABC,815.000,199577.00000,974.000,937.000,884.000,931.666667,45.236416,7936.000,992.000000,837.000,1256.000,37.000,-325.000,246984.682000,218668.693000,241963.000000,235872.125000,15108.702028,1.950837e+06,243854.610752,199145.596010,277904.838000,28315.989000,-325.000,1,0,0,0.0,0.0,0.0
49,2021-11-09,Beverages,XYZ,971.000,263813.85400,677.000,1043.000,1086.000,935.333333,224.753940,7172.000,896.500000,677.000,1086.000,-366.000,49.000,201386.081000,289993.000000,262426.080000,251268.387000,45344.978867,1.877084e+06,234635.446128,187931.769020,289993.000000,-88606.919000,49.000,1,0,0,1.0,0.0,0.0
50,2021-11-09,Grocery,ABC,2489.046,497413.33899,2326.638,2646.820,2614.006,2529.154667,176.150333,20427.563,2553.445375,2155.128,3021.684,-320.182,-532.638,438841.438010,505512.742970,548660.900000,497671.693660,55328.022725,3.912755e+06,489094.381872,397946.965024,582206.592985,-66671.304960,-532.638,1,0,0,0.0,1.0,0.0
51,2021-11-09,Grocery,XYZ,3608.112,622938.06900,2879.414,3608.962,3625.864,3371.413333,426.167722,26569.025,3321.128125,2879.414,3625.864,-729.548,-15.604,498037.791000,661655.000010,622856.562004,594183.117671,85494.283894,4.508435e+06,563554.392877,491450.723000,661655.000010,-163617.209010,-15.604,1,0,0,1.0,1.0,0.0


In [10]:
#creating train and test set from preprocessed data
from utils.model_helpers import create_training_testing
train_dict, test_dict = create_training_testing(full_df_processed, test_date_start='2022-02-01')

In [11]:
#Setting training and testing data
train_X = train_dict['train_features']
train_y_sales = train_dict['train_net_sales']
train_y_item_qty = train_dict['train_item_qty']

test_X = test_dict['train_features']
test_y_sales = test_dict['train_net_sales']
test_y_item_qty = test_dict['train_item_qty']

In [12]:
train_X

,lag_item_qty_1,lag_item_qty_2,lag_item_qty_3,rolling_mean_item_qty_3,rolling_std_item_qty_3,cumsum_item_qty,cummean_item_qty,expanding_min_item_qty,expanding_max_item_qty,diff_item_qty,diff_item_qty_7,lag_net_sales_1,lag_net_sales_2,lag_net_sales_3,rolling_mean_net_sales_3,rolling_std_net_sales_3,cumsum_net_sales,cummean_net_sales,expanding_min_net_sales,expanding_max_net_sales,diff_net_sales,diff_net_sales_7,day_of_week,isWeekend,Is_Holiday,store_XYZ,item_dept_Grocery,item_dept_Household
42,937.000,884.000,991.000,937.333333,53.500779,6962.000,994.571429,837.000,1256.000,53.000,57.000,218668.693000,241963.000000,255695.36500,238775.686000,18717.981669,1.703852e+06,243407.457717,199145.596010,2.779048e+05,-23294.307000,57.000,0,0,0,0.0,0.0,0.0
43,1043.000,1086.000,910.000,1013.000000,91.755109,6495.000,927.857143,779.000,1086.000,-43.000,-102.000,289993.000000,262426.080000,217746.00000,256721.693333,36459.734951,1.675697e+06,239385.355431,187931.769020,2.899930e+05,27566.920000,-102.000,0,0,0,1.0,0.0,0.0
44,2646.820,2614.006,2654.582,2638.469333,21.538411,18100.925,2585.846429,2155.128,3021.684,32.814,-33.761,505512.742970,548660.900000,479383.66397,511185.768980,34985.300857,3.473914e+06,496273.373853,397946.965024,5.822066e+05,-43148.157030,-33.761,0,0,0,0.0,1.0,0.0
45,3608.962,3625.864,3287.828,3507.551333,190.473559,23689.611,3384.230143,2987.060,3625.864,-16.902,-107.646,661655.000010,622856.562004,557072.72900,613861.430338,52868.205452,4.010397e+06,572913.907431,491450.723000,6.616550e+05,38798.438006,-107.646,0,0,0,1.0,1.0,0.0
46,998.000,1152.000,986.000,1045.333333,92.570694,7114.000,1016.285714,819.000,1195.000,-154.000,-160.000,233781.880016,284326.000000,265589.69202,261232.524012,25552.215188,1.775416e+06,253630.794154,209100.513996,2.920205e+05,-50544.119984,-160.000,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,1048.000,1255.000,1336.000,1213.000000,148.522726,92683.000,1018.494505,645.000,1761.000,-207.000,22.000,239877.764000,337437.000000,513057.00000,363457.254667,138435.956258,2.381819e+07,261738.336506,149946.090000,5.130570e+05,-97559.236000,22.000,0,0,0,1.0,0.0,0.0
548,2869.215,2587.436,2358.758,2605.136333,255.688411,236593.356,2599.926989,1779.437,7787.289,281.779,286.094,585570.036980,551861.601020,464186.11400,533872.584000,62659.538296,4.889516e+07,537309.461967,367440.183000,2.035445e+06,33708.435960,286.094,0,0,0,0.0,1.0,0.0
549,4040.207,3539.002,3162.792,3580.667000,440.188880,306311.340,3366.058681,2396.285,5963.381,501.205,434.602,756342.462000,667129.626000,583466.77600,668979.621333,86452.689747,5.545075e+07,609348.883327,443513.481000,1.161787e+06,89212.836000,434.602,0,0,0,1.0,1.0,0.0
550,1170.000,1236.000,1062.000,1156.000000,87.840765,94915.000,1043.021978,718.000,1606.000,-66.000,180.000,323308.267970,349645.837880,290746.99899,321233.701613,29504.172160,2.512344e+07,276081.704621,181968.999000,4.518945e+05,-26337.569910,180.000,0,0,0,0.0,0.0,1.0


In [13]:
test_X

,lag_item_qty_1,lag_item_qty_2,lag_item_qty_3,rolling_mean_item_qty_3,rolling_std_item_qty_3,cumsum_item_qty,cummean_item_qty,expanding_min_item_qty,expanding_max_item_qty,diff_item_qty,diff_item_qty_7,lag_net_sales_1,lag_net_sales_2,lag_net_sales_3,rolling_mean_net_sales_3,rolling_std_net_sales_3,cumsum_net_sales,cummean_net_sales,expanding_min_net_sales,expanding_max_net_sales,diff_net_sales,diff_net_sales_7,day_of_week,isWeekend,Is_Holiday,store_XYZ,item_dept_Grocery,item_dept_Household
552,887.000,923.000,966.000,925.333333,39.551654,102220.000,1111.086957,756.000,1748.000,-36.000,798.000,2.046880e+05,196153.950000,213541.730000,2.047945e+05,8694.380072,2.626751e+07,285516.456804,168349.26000,5.329314e+05,8534.001010,798.000,1,0,0,0.0,0.0,0.0
553,791.000,1048.000,1255.000,1031.333333,232.448561,93474.000,1016.021739,645.000,1761.000,-257.000,266.000,1.791224e+05,239877.764000,337437.000000,2.521457e+05,79867.125785,2.399731e+07,260840.336816,149946.09000,5.130570e+05,-60755.398990,266.000,1,0,0,1.0,0.0,0.0
554,2603.874,2869.215,2587.436,2686.841667,158.153648,239197.230,2599.969891,1779.437,7787.289,-265.341,753.007,5.426439e+05,585570.036980,551861.601020,5.600252e+05,22597.472831,4.943780e+07,537367.445327,367440.18300,2.035445e+06,-42926.105980,753.007,1,0,0,0.0,1.0,0.0
555,2996.909,4040.207,3539.002,3525.372667,521.782520,309308.249,3362.046185,2396.285,5963.381,-1043.298,469.890,5.630437e+05,756342.462000,667129.626000,6.621719e+05,96744.682254,5.601379e+07,608845.566486,443513.48100,1.161787e+06,-193298.728000,469.890,1,0,0,1.0,1.0,0.0
556,1115.000,1170.000,1236.000,1173.666667,60.583276,96030.000,1043.804348,718.000,1606.000,-55.000,235.000,3.148003e+05,323308.267970,349645.837880,3.292515e+05,18167.127914,2.543824e+07,276502.558593,181968.99900,4.518945e+05,-8507.997980,235.000,1,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,1751.000,1126.000,1317.000,1398.000000,320.276443,130083.000,1093.134454,645.000,1763.000,625.000,114.000,5.310318e+05,269641.753000,396324.003000,3.989992e+05,130715.547043,3.410990e+07,286637.847235,149946.09000,5.813210e+05,261390.028980,114.000,0,0,0,1.0,0.0,0.0
716,3752.998,3486.773,4293.282,3844.351000,410.941869,325810.103,2737.900025,1779.437,7787.289,266.225,868.876,7.867650e+05,725599.851278,867613.506013,7.933261e+05,71233.808433,6.772851e+07,569147.160637,367440.18300,2.035445e+06,61165.191660,868.876,0,0,0,0.0,1.0,0.0
717,7599.386,5254.694,4246.995,5700.358333,1720.056359,426823.943,3586.755824,2396.285,7599.386,2344.692,1239.090,1.596598e+06,985413.796996,805993.474911,1.129335e+06,414486.318350,7.845215e+07,659261.805316,443513.48100,1.596598e+06,611184.257052,1239.090,0,0,0,1.0,1.0,0.0
718,1642.000,1508.000,1750.000,1633.333333,121.232559,131131.000,1101.941176,718.000,1750.000,134.000,275.000,4.441275e+05,390277.067990,453002.517000,4.291357e+05,33943.864780,3.497245e+07,293886.126171,181968.99900,4.530025e+05,53850.418996,275.000,0,0,0,0.0,0.0,1.0


### Modelling

In [14]:
from utils.SalesItemQtyModel import SalesItemQtyModel

In [15]:
#Need to define 2 models to predict sales and item_qty
from catboost import CatBoostRegressor

#params chosen from hp tuning
params = {'bagging_temperature': 0.5447824858171504,'border_count': 220.0,'depth': 5.0,'iterations': 800.0,
          'l2_leaf_reg': 13.06135266464145,'learning_rate': 0.15538685598864,'one_hot_max_size': 34.0,
          'random_strength': 4.115171860221108}

sales_model = CatBoostRegressor(**params, verbose = 0)
item_qty_model = CatBoostRegressor(**params, verbose = 0)

In [16]:
#Defining the model to train and predict both sales and item qty
sales_item_qty_model = SalesItemQtyModel(
    model_sales=sales_model,
    model_item_qty=item_qty_model
)

In [17]:
#Fitting the model
sales_item_qty_model.fit(X = train_X, y_sales = train_y_sales, y_item_qty = train_y_item_qty)

SalesItemQtyModel(model_item_qty=<catboost.core.CatBoostRegressor object at 0x7f8128b52590>,
                  model_sales=<catboost.core.CatBoostRegressor object at 0x7f8128b52740>)

In [18]:
pd.DataFrame({'feature': train_X.columns,
'importance_sales': sales_item_qty_model.model_sales.feature_importances_}).sort_values(by='importance_sales', ascending=False)

,feature,importance_sales
17,cummean_net_sales,15.471971
10,diff_item_qty_7,11.782388
6,cummean_item_qty,9.538189
21,diff_net_sales_7,9.045278
22,day_of_week,6.441875
0,lag_item_qty_1,5.578214
18,expanding_min_net_sales,3.594095
11,lag_net_sales_1,3.468588
3,rolling_mean_item_qty_3,3.315649
2,lag_item_qty_3,3.047510


In [19]:
pd.DataFrame({'feature': train_X.columns,
              'importance_item_qty': sales_item_qty_model.model_item_qty.feature_importances_}).sort_values(by='importance_item_qty', ascending=False)

,feature,importance_item_qty
6,cummean_item_qty,16.689890
0,lag_item_qty_1,11.934048
10,diff_item_qty_7,8.336585
21,diff_net_sales_7,7.902453
1,lag_item_qty_2,5.703436
22,day_of_week,5.335345
7,expanding_min_item_qty,4.784200
3,rolling_mean_item_qty_3,4.459325
8,expanding_max_item_qty,4.259095
2,lag_item_qty_3,4.065018


In [20]:
from pickle import dump
with open("../models/final_model.pkl", "wb") as f:
    dump(sales_item_qty_model, f, protocol=5)

In [21]:
with open("../models/preprocessor.pkl", "wb") as f:
    dump(preprocessor, f, protocol=5)

### Model Testing

In [22]:
from utils.model_helpers import get_results_fitted
get_results_fitted(train_dict=train_dict, valid_dict=test_dict, sales_item_qty_model=sales_item_qty_model)

Train Set Results...

MAPE for predicting Sales: 0.02100911588366927
MAPE for predicting Item Qty: 0.015380185311012639

Test Set Results...

MAPE for predicting Sales: 0.18667983170506036
MAPE for predicting Item Qty: 0.1455216681566889



### Production Level Forecasting

In [23]:
#This is to get the predictions in a real scenario (by forecasting using only the training set)

In [24]:
train_df_gb = preprocessor._groupby_df(train_df)
test_df_gb = preprocessor._groupby_df(test_df)
depts_list = train_df_gb['item_dept'].unique()
stores_list = train_df_gb['store'].unique()

In [25]:
from utils.model_helpers import generate_forecasting_df
feb_forecasted_df = generate_forecasting_df(start_date='2022-02-01', end_date='2022-02-28', depts_list=depts_list,
                                            stores_list=stores_list,historical_df=train_df_gb, dual_model=sales_item_qty_model,
                                            preprocessor=preprocessor)

In [26]:
feb_forecasted_df

,date_id,item_dept,store,net_sales,item_qty
0,2022-02-01,Beverages,ABC,256286.176582,1007.133387
1,2022-02-01,Beverages,XYZ,222458.745834,898.002637
2,2022-02-01,Grocery,ABC,697563.119719,3086.222626
3,2022-02-01,Grocery,XYZ,683009.846470,3512.437157
4,2022-02-01,Household,ABC,274217.838980,1202.294195
...,...,...,...,...,...
163,2022-02-28,Beverages,XYZ,216969.282702,948.449825
164,2022-02-28,Grocery,ABC,588028.037395,2995.321694
165,2022-02-28,Grocery,XYZ,684538.626587,3369.106483
166,2022-02-28,Household,ABC,261885.076127,1159.244110


In [27]:
from utils.model_helpers import store_level_results
store_level_results(test_df=test_df_gb, pred_df=feb_forecasted_df)

Overall MAPE score for Sales: 0.1752244618107001
Overall MAPE score for Item Qty: 0.12352416383621237

For Store ABC, MAPE for predicting sales each day: 0.14691811769396676
For Store ABC, MAPE for predicting item qty each day: 0.09502702391600983

For Store ABC and Department Beverages, MAPE for predicting sales each day: 0.30097013681603224
For Store ABC and Department Beverages, MAPE for predicting item qty each day: 0.21467199631040315

For Store ABC and Department Grocery, MAPE for predicting sales each day: 0.1059989702921912
For Store ABC and Department Grocery, MAPE for predicting item qty each day: 0.11472500064623288

For Store ABC and Department Household, MAPE for predicting sales each day: 0.20574111796168432
For Store ABC and Department Household, MAPE for predicting item qty each day: 0.11368949262734322

For Store XYZ, MAPE for predicting sales each day: 0.2035308059274334
For Store XYZ, MAPE for predicting item qty each day: 0.1520213037564149

For Store XYZ and Depart

In [26]:
test_y_sales.shape, test_y_item_qty.shape

((168,), (168,))

In [27]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(test_y_sales, feb_forecasted_df['net_sales'])

0.21309893455679807

In [28]:
mean_absolute_percentage_error(test_y_item_qty, feb_forecasted_df['item_qty'])

0.1549441127409602

##### GroupBy Store: Monitoring Store Performance

In [29]:
feb_forecasted_df_stores = feb_forecasted_df.groupby(['date_id', 'store'])[['net_sales', 'item_qty']].sum().reset_index()

In [30]:
feb_forecasted_df.head(10)

,date_id,item_dept,store,net_sales,item_qty
0,2022-02-01,Beverages,ABC,256286.176582,1007.133387
1,2022-02-01,Beverages,XYZ,222458.745834,898.002637
2,2022-02-01,Grocery,ABC,697563.119719,3086.222626
3,2022-02-01,Grocery,XYZ,683009.846470,3512.437157
4,2022-02-01,Household,ABC,274217.838980,1202.294195
5,2022-02-01,Household,XYZ,247067.375887,1160.606818
6,2022-02-02,Beverages,ABC,282756.963205,1101.433435
7,2022-02-02,Beverages,XYZ,271272.579183,1147.224903
8,2022-02-02,Grocery,ABC,601553.938929,2936.461767
9,2022-02-02,Grocery,XYZ,595255.987210,3270.000402


In [31]:
feb_forecasted_df_stores.head(10)

,date_id,store,net_sales,item_qty
0,2022-02-01,ABC,1.228067e+06,5295.650208
1,2022-02-01,XYZ,1.152536e+06,5571.046611
2,2022-02-02,ABC,1.213581e+06,5306.568708
3,2022-02-02,XYZ,1.095721e+06,5377.643121
4,2022-02-03,ABC,1.480473e+06,6131.762084
5,2022-02-03,XYZ,1.288443e+06,6050.982359
6,2022-02-04,ABC,1.206545e+06,5660.627848
7,2022-02-04,XYZ,1.206220e+06,5957.669810
8,2022-02-05,ABC,1.248368e+06,5804.939319
9,2022-02-05,XYZ,1.342932e+06,6639.093907


In [32]:
test_df_stores = test_df.groupby(['date_id', 'store'])[['net_sales', 'item_qty']].sum().reset_index()
test_df_stores

,date_id,store,net_sales,item_qty
0,2022-02-01,ABC,1.516543e+06,6517.128
1,2022-02-01,XYZ,1.389440e+06,6623.394
2,2022-02-02,ABC,1.444364e+06,6340.287
3,2022-02-02,XYZ,1.557627e+06,7188.348
4,2022-02-03,ABC,1.756087e+06,7445.199
5,2022-02-03,XYZ,1.642178e+06,7523.119
6,2022-02-04,ABC,1.388147e+06,5023.904
7,2022-02-04,XYZ,1.802251e+06,7743.322
8,2022-02-05,ABC,1.536005e+06,5849.920
9,2022-02-05,XYZ,1.705150e+06,7182.791


In [33]:
feb_forecasted_df_stores

,date_id,store,net_sales,item_qty
0,2022-02-01,ABC,1.228067e+06,5295.650208
1,2022-02-01,XYZ,1.152536e+06,5571.046611
2,2022-02-02,ABC,1.213581e+06,5306.568708
3,2022-02-02,XYZ,1.095721e+06,5377.643121
4,2022-02-03,ABC,1.480473e+06,6131.762084
5,2022-02-03,XYZ,1.288443e+06,6050.982359
6,2022-02-04,ABC,1.206545e+06,5660.627848
7,2022-02-04,XYZ,1.206220e+06,5957.669810
8,2022-02-05,ABC,1.248368e+06,5804.939319
9,2022-02-05,XYZ,1.342932e+06,6639.093907


In [34]:
mean_absolute_percentage_error(test_df_stores['net_sales'], feb_forecasted_df_stores['net_sales'])

0.1752244618107001

In [35]:
mean_absolute_percentage_error(test_df_stores['item_qty'], feb_forecasted_df_stores['item_qty'])

0.12352416383621234

In [36]:
test_df_stores[test_df_stores['store'] == 'ABC']

,date_id,store,net_sales,item_qty
0,2022-02-01,ABC,1.516543e+06,6517.128
2,2022-02-02,ABC,1.444364e+06,6340.287
4,2022-02-03,ABC,1.756087e+06,7445.199
6,2022-02-04,ABC,1.388147e+06,5023.904
8,2022-02-05,ABC,1.536005e+06,5849.920
10,2022-02-06,ABC,1.171293e+06,5273.912
12,2022-02-07,ABC,1.257475e+06,5819.820
14,2022-02-08,ABC,1.367424e+06,5876.401
16,2022-02-09,ABC,1.319227e+06,5564.455
18,2022-02-10,ABC,1.568063e+06,6605.404


In [37]:
feb_forecasted_df_stores[feb_forecasted_df_stores['store'] == 'ABC']

,date_id,store,net_sales,item_qty
0,2022-02-01,ABC,1.228067e+06,5295.650208
2,2022-02-02,ABC,1.213581e+06,5306.568708
4,2022-02-03,ABC,1.480473e+06,6131.762084
6,2022-02-04,ABC,1.206545e+06,5660.627848
8,2022-02-05,ABC,1.248368e+06,5804.939319
10,2022-02-06,ABC,1.178523e+06,5652.744628
12,2022-02-07,ABC,1.248778e+06,5468.659786
14,2022-02-08,ABC,1.171436e+06,5507.452542
16,2022-02-09,ABC,1.305258e+06,5761.024433
18,2022-02-10,ABC,1.234707e+06,6001.671214
